### Elementos de salida de  cv.calibrateCamera()

- ret:
- mtx:   Matriz intrínseca de cámara de coma flotante de 3x3 de entrada/salida
- dist:  Vector de entrada/salida de coeficientes de distorsión
- rvecs: Vector de salida de vectores de rotación
- tvecs: Vector de salida de los vectores de traducción estimados para cada vista de patrón, consulte la descripción de parámetros anterior.

In [21]:
import numpy as np
import cv2 as cv
import glob

class Parameters:
    ret   = None
    mtx   = None
    dist  = None
    rvecs = None
    tvecs = None
    def __init__(self,ret, mtx, dist, rvecs, tvecs):
        self.ret   = ret
        self.mtx   = mtx
        self.dist  = dist
        self.rvecs = rvecs
        self.tvecs = tvecs

In [22]:

def calibrate(path_image,size_chessboard):
    # termination criteria
    criteria = (cv.TERM_CRITERIA_EPS + cv.TERM_CRITERIA_MAX_ITER, int(size_chessboard), 0.001)
    a,b=9,6
    # prepare object points, like (0,0,0), (1,0,0), (2,0,0) ....,(6,5,0)
    objp = np.zeros((a*b,3), np.float32)
    objp[:,:2] = np.mgrid[0:a,0:b].T.reshape(-1,2)
    # Arrays to store object points and image points from all the images.
    objpoints = [] # 3d point in real world space
    imgpoints = [] # 2d points in image plane.
    img = cv.imread(path_image)
    gray = cv.cvtColor(img, cv.COLOR_BGR2GRAY)
    # Find the chess board corners
    ret, corners = cv.findChessboardCorners(img, (a,b), None)
    # If found, add object points, image points (after refining them)
    if ret == True:
        objpoints.append(objp)
        corners = cv.cornerSubPix(gray,corners, (a,b), (-1,-1), criteria)
        imgpoints.append(corners)
    ret, mtx, dist, rvecs, tvecs = cv.calibrateCamera(objpoints, imgpoints, gray.shape[::-1], None, None)
    return Parameters(ret, mtx, dist, rvecs, tvecs)

In [23]:
def radial_correction(path,path_out,parameters):
    img = cv.imread(path)
    h,  w = img.shape[:2]
    newcameramtx, roi = cv.getOptimalNewCameraMatrix(parameters.mtx, parameters.dist, (w,h), 1, (w,h))
    print(roi)
    # undistort
    dst = cv.undistort(img, parameters.mtx, parameters.dist, None, newcameramtx)
    # crop the image
    x, y, w, h = roi
    #x,y=0,0
    dst = dst[y:y+h, x:x+w]
    cv.imwrite(path_out, dst)

In [24]:
def radial_correction_beta(path,path_out,parameters):
    img = cv.imread(path)
    h,  w = img.shape[:2]
    newcameramtx, roi = cv.getOptimalNewCameraMatrix(parameters.mtx, parameters.dist, (w,h), 1, (w,h))
    print(roi)
    # undistort
    mapx, mapy = cv.initUndistortRectifyMap(parameters.mtx, parameters.dist, None, newcameramtx, (w,h), 5)
    dst = cv.remap(img, mapx, mapy, cv.INTER_LINEAR)
    # crop the image
    x, y, w, h = roi
    dst = dst[y:y+h, x:x+w]
    cv.imwrite(path_out, dst)

# Test

In [25]:
path='img/images/image_0.jpg'
parameters=calibrate(path,10)
print(parameters.mtx)


[[832.94274257   0.         402.50332051]
 [  0.         834.8724402  223.29354526]
 [  0.           0.           1.        ]]


In [26]:
path='img/images/far/image_0.jpg'
path_out='img/calibresult_1.png'
radial_correction(path,path_out,parameters)

(136, 38, 402, 274)


In [27]:
path='img/images/far/image_0.jpg'
path_out='img/calibresult_2.png'
radial_correction_beta(path,path_out,parameters)

(136, 38, 402, 274)
